run edge attribution patching on hex
fit multivariate gaussian to attribution scores
use malanabois from trusted to untrusted

need to figure out which edges to monitor, I guess all MLP and attention for now? 
in tampering case we can maybe only use streams on the measurment tokens

# Run Edge Attribution On ARC Tasks

In [1]:
import json

import torch 
import einops
from jaxtyping import Float

from eap.eap_wrapper import EAP
from cupbearer import data, detectors, models, scripts, tasks, utils
from transformer_lens import HookedTransformer


/Users/oliverdaniels-koch/Library/Caches/pypoetry/virtualenvs/elk-experiments-AZ2LBS3Q-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Model and Data Setup

In [2]:
task_name = "hex"
device = "mps"

In [3]:
model = HookedTransformer.from_pretrained(
    "attn-only-2l",
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    fold_value_biases=False,
).to(device)

Loaded pretrained model attn-only-2l into HookedTransformer
Moving model to device:  mps


In [4]:
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

In [5]:
task = tasks.tiny_natural_mechanisms(task_name, device)

Loaded pretrained model attn-only-1l into HookedTransformer
Moving model to device:  mps


In [6]:
# model = task.model 

# # model.set_use_hook_mlp_in(True)
# model.set_use_split_qkv_input(True)
# model.set_use_attn_result(True)

## Metric Setup

In [7]:
with open(f".cupbearer_cache/tiny_natural_mechanisms/{task_name}_task.json") as f:
    task_config = json.load(f)

# cpu to avoid MPS issues with tensors that are too large
effect_tokens = torch.tensor(
    task_config["effect_tokens"], dtype=torch.long, device="cpu"
)

cause_tokens = torch.tensor(
    task_config["cause_tokens"], dtype=torch.long, device="cpu"
)

In [8]:
# use sum probability over "effect" tokens (intended behavior)
def effect_prob_func(logits):
    assert logits.ndim == 3
    probs = logits.softmax(-1)
    # Sum over vocab and batch dim (for now we're just computing attribution values, we'll deal with per data instance later)
    out = probs[:, -1, effect_tokens].sum()
    # print("metric", out)
    return out

## Run Experiment

Going to need to define detector, but for now lets just get it running

In [9]:
# tokenize data in batches
trusted_tokens = torch.stack([torch.tensor(x["prefix_tokens"]) for x in task.trusted_data.data], dim=0).to(device)

In [10]:
trusted_tokens.shape, trusted_tokens.shape[0] / 64

(torch.Size([16384, 16]), 256.0)

In [11]:
model.reset_hooks()
# The problem is that this is a one-layer attention only model
# TODO: extend EAP to QK OV?
# or just try this on a larger model
graph = EAP(
    model=model,
    clean_tokens=trusted_tokens,
    metric=effect_prob_func, 
    upstream_nodes=["head"],
    downstream_nodes=["head"],
    batch_size=64
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Saving activations requires 0.0000 GB of memory per token


  0%|          | 0/256 [00:00<?, ?it/s]/Users/oliverdaniels-koch/Library/Caches/pypoetry/virtualenvs/elk-experiments-AZ2LBS3Q-py3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


running backwards hook


  0%|          | 1/256 [00:01<08:25,  1.98s/it]

...

running backwards hook
running backwards hook
running backwards hook
running backwards hook
running backwards hook
running backwards hook


100%|█████████▉| 255/256 [05:07<00:01,  1.19s/it]

running backwards hook
running backwards hook
running backwards hook
running backwards hook
running backwards hook
running backwards hook


100%|██████████| 256/256 [05:08<00:00,  1.21s/it]

running backwards hook
running backwards hook
running backwards hook
running backwards hook
running backwards hook
running backwards hook


In [ ]:
# I guess maybe stay away from transformer lens until automatic huggingface stuff is supported?
# seems like a shame
# alternatively could 
    # figure how to load the base model state dict into transformer lens
    # factor out the probes, such that you could easily load the probes seperately and use them for a metric



# use the transformer lens version as the base model 
# use

In [18]:
top_edges = graph.top_edges(n=20, abs_scores=True)

In [19]:
for from_edge, to_edge, score in top_edges:
    print(f'{from_edge} -> [{round(score, 3)}] -> {to_edge}')

head.0.0 -> [0.095] -> head.1.6.k
head.0.3 -> [-0.07] -> head.1.6.k
head.0.4 -> [-0.067] -> head.1.6.k
head.0.2 -> [-0.022] -> head.1.6.v
head.0.1 -> [-0.022] -> head.1.6.v
head.0.3 -> [-0.018] -> head.1.6.v
head.0.4 -> [-0.017] -> head.1.6.q
head.0.5 -> [-0.016] -> head.1.4.v
head.0.2 -> [-0.014] -> head.1.5.v
head.0.1 -> [0.014] -> head.1.6.k
head.0.2 -> [-0.014] -> head.1.6.q
head.0.1 -> [-0.013] -> head.1.6.q
head.0.2 -> [-0.012] -> head.1.6.k
head.0.0 -> [-0.011] -> head.1.4.k
head.0.1 -> [-0.011] -> head.1.4.v
head.0.1 -> [-0.011] -> head.1.4.q
head.0.2 -> [-0.011] -> head.1.2.v
head.0.1 -> [0.01] -> head.1.2.v
head.0.3 -> [0.01] -> head.1.5.q
head.0.6 -> [-0.01] -> head.1.3.v


In [ ]:
# TODO: follow notebook setup but using arc task model
